In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor, PrintMonitor
from mand.core import find, addFootnote, setDependencyManager
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
class C(Entity):
    
    @node
    def fib(self, n):
        if n < 2:
            return n
        return self.fib(n-1) + self.fib(n-2)


In [3]:
c = C()

In [4]:
with SummaryMonitor():
    v = c.fib(10)
print v

with SummaryMonitor():
    v = c.fib(40)
print v

# Compute activity summary (0.00 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|19
|GetValue/Calc|11

55


# Compute activity summary (0.00 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|61
|GetValue/Calc|30

102334155


In [5]:
class D(Entity):
    
    @node(tweakable=True)
    def v(self): 
        return 10
        
class E(D):
    
    @node(tweakable=True)
    def v(self): 
        return super(E, self).v() + 20
    
e = E()

print 'Expect 30:', e.v()
assert e.v() == 30

with Context({e.v: 40}):
    print 'Expect 40:', e.v()
    assert e.v() == 40
        
print 'Expect 30:', e.v()
assert e.v() == 30

with Context({super(E, e).v: 40}):
    print 'Expect 60:', e.v()
    assert e.v() == 60

Expect 30: 30
Expect 40: 40
Expect 30: 30
Expect 60: 60


In [6]:
from mand.demos.trading import MarketDataSource

class MarketInterface1(Entity):
    
    @node(tweakable=True)
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node(tweakable=True)
    def spot(self):
        return self.source().last()
      
_tr.add(MarketInterface1)

with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_ibm.update(last=175.61)

    s2_ibm  = MarketDataSource('user_marks.IBM')
    s2_ibm.update(last=180)
    
    ibm  = MarketInterface1('IBM').write()
    
print ibm.spot()
with Context({ibm.sourceName: 'user_marks'}):
    print ibm.spot()
print ibm.spot()

175.61
180
175.61


In [7]:
from mand.demos.depmanager import DM1

class DM2(DM1):
    
    def __init__(self):
        super(DM2, self).__init__()
        #self.simpleClockMethods = ('Clock:cutoffs', 'Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items') # , 'MarketInterface1:spot')
        self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items') # , 'MarketInterface1:spot')
        #self.simpleClockMethods = ('Clock:cutoffs', 'Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items', 'MarketInterface1:spot')
        
    def calculated(self, node):
        if not node.isSimplified:
            return
        print 'CALCULATED:', node
        for input in node.inputs:
            print '  INPUT:', input
        super(DM1, self).calculated(node)
                
dm = DM2()
setDependencyManager(dm)

db4 = db.copy()   
ibm2 = db4.get(ibm.meta.path())

ts = Timestamp()


with Context({ibm2.sourceName: 'user_marks'}, 'user'):
    with SummaryMonitor():
        v = ibm2.spot()
    print v
    node = getNode(ibm2.spot)
    node.printInputGraph()

footnotes = node.footnotes.values()
print footnotes

for k, v in dm.contexts.items():
    print '%s: %s' % (k, v)

print 

with SummaryMonitor():
    v = ibm2.spot()
print v
node = getNode(ibm2.spot)
node.printInputGraph()
footnotes = node.footnotes.values()
print footnotes


CALCULATED: <Node: /Global/MarketDataSource/user_marks.IBM.(RefData:state)() in Root:user:Simp-MarketData @4463224336 T=False>
  INPUT: <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:user:Simp-MarketData @4463224208 T=True>


# Compute activity summary (0.04 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|6
|GetValue|29
|GetValue/Calc|15

180
<Node: /Global/MarketInterface1/IBM.(MarketInterface1:spot)() in Root:user @4463074064 T=True>, nIn=2
   <Node: /Global/MarketDataSource/user_marks.IBM.(ExternalRefData:state)() in Root:user @4463124752 T=False>, nIn=2
      <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:user @4463074704 T=True>, nIn=2
         <Node: /Global/Clock/MarketData.(Clock:parent)() in Root:user @4463075216 T=False>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:user @4463127312 T=True>, nIn=0
         <Node: /Global/Clock/RefData.(Clock:cutoffs)() in Root:user @4463222992 T=True>, nIn=2
            <Node: /Global/Clock/RefData.(Clock:parent)() in Root:user @4463223376 T=False>, nIn=1
               <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:user @4463127312 T=True>, nIn=0
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:user @4463127312 T=True>, nIn=0
      <Node: /Global/MarketDataSource/user_marks.IBM.(RefData:state)() i

# Compute activity summary (0.02 seconds of wall clock time)

|key|value|
|-|-|
|Db.Get|2
|GetValue|29
|GetValue/Calc|15

175.61
<Node: /Global/MarketInterface1/IBM.(MarketInterface1:spot)() in Root @4463222864 T=True>, nIn=2
   <Node: /Global/MarketDataSource/source1.IBM.(ExternalRefData:state)() in Root @4463127056 T=False>, nIn=2
      <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root @4463223760 T=True>, nIn=2
         <Node: /Global/Clock/MarketData.(Clock:parent)() in Root @4462902032 T=False>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root @4462901456 T=True>, nIn=0
         <Node: /Global/Clock/RefData.(Clock:cutoffs)() in Root @4463073232 T=True>, nIn=2
            <Node: /Global/Clock/RefData.(Clock:parent)() in Root @4463276304 T=False>, nIn=1
               <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root @4462901456 T=True>, nIn=0
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root @4462901456 T=True>, nIn=0
      <Node: /Global/MarketDataSource/source1.IBM.(RefData:state)() in Root:user:Simp-MarketData @4463277392 T=False>